# Camada Gold (Business / Aggregated Layer)

Dados prontos para consumo, com agregações, sumarizações e cálculos.

Normalmente estruturados para atender demandas específicas do negócio.

Utilizado por dashboards, relatórios e aplicações finais.

In [0]:
df_silver = spark.table("pipe1.silver.slv_despesas_timbauvas")
display(df_silver)

In [0]:
df_silver = spark.table("pipe1.silver.slv_despesas_lcb_contemporaneo")
display(df_silver)

#Código PySpark para criar a tabela Gold agregando as duas fontes:

In [0]:
from pyspark.sql.functions import lit

# 1. Ler as tabelas Silver
df_timbauvas = spark.table("pipe1.silver.slv_despesas_timbauvas")
df_lcb = spark.table("pipe1.silver.slv_despesas_lcb_contemporaneo")

# 2. Adicionar a coluna de origem
df_timbauvas = df_timbauvas.withColumn("origem", lit("Timbauvas"))
df_lcb = df_lcb.withColumn("origem", lit("LCB"))

# 3. Criar views temporárias
df_timbauvas.createOrReplaceTempView("vw_timbauvas")
df_lcb.createOrReplaceTempView("vw_lcb")

# 4. Fazer o UNION ALL com padronização
query = """
SELECT 
    periodo,
    ano,
    mes,
    origem,
    despesa_total_condominio,
    fundo_pintura,
    leitura_medidor_gas,
    fundo_reserva,
    bandeira,
    preco_kg_gas,
    extras,
    valor_gas,
    internet_cond,
    taxa_condominio,
    despesa_de_luz,
    consumo_kg,
    agua_consumo,
    valor_servico,
    agua_area_comum,
    consumo_gas_m_3,
    valor_em_m_3,
    consumo_de_agua_m_3,
    kwh_dia,
    agua_tarifa,
    fator_conversao,
    gas,
    leitura_medidor_luz,
    dias_consumo,
    despesa_bancaria,
    kwh,
    dia_consumido,
    total_agua,
    _fivetran_synced
FROM vw_lcb

UNION ALL

SELECT 
    periodo,
    ano,
    mes,
    origem,
    despesa_total_condominio,
    NULL AS fundo_pintura,
    NULL AS leitura_medidor_gas,
    CAST(fundo_reserva AS DOUBLE) AS fundo_reserva,
    bandeira,
    NULL AS preco_kg_gas,
    extras,
    NULL AS valor_gas,
    NULL AS internet_cond,
    taxa_condominio,    
    despesa_de_luz,
    NULL AS consumo_kg,
    agua_consumo,
    NULL AS valor_servico,
    NULL AS agua_area_comum,
    NULL AS consumo_gas_m_3,
    valor_em_m_3,
    NULL AS consumo_de_agua_m_3,
    kwh_dia,
    agua_tarifa,
    NULL AS fator_conversao,
    CAST(gas AS DOUBLE) AS gas,
    NULL AS leitura_medidor_luz,
    NULL AS dias_consumo,
    NULL AS despesa_bancaria,
    kwh,
    dia_consumido,
    total_agua,
    _fivetran_synced
FROM vw_timbauvas
"""

# 5. Executar a query
df_gold = spark.sql(query)

# 6. Reordenar as colunas manualmente
df_gold = df_gold.select(
    "periodo",
    "ano",
    "mes",
    "origem",
    "despesa_total_condominio",
    "fundo_pintura",
    "leitura_medidor_gas",
    "fundo_reserva",
    "bandeira",
    "preco_kg_gas",
    "extras",
    "valor_gas",
    "internet_cond",
    "taxa_condominio",
    "despesa_de_luz",
    "consumo_kg",
    "agua_consumo",
    "valor_servico",
    "agua_area_comum",
    "consumo_gas_m_3",
    "valor_em_m_3",
    "consumo_de_agua_m_3",
    "kwh_dia",
    "agua_tarifa",
    "fator_conversao",
    "gas",
    "leitura_medidor_luz",
    "dias_consumo",
    "despesa_bancaria",
    "kwh",
    "dia_consumido",
    "total_agua",
    "_fivetran_synced"
)

# 7. Escrever particionado por ano e mês
df_gold.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("ano", "mes") \
    .saveAsTable("pipe1.gold.gld_despesas_unificadas")


In [0]:
df_gold = spark.table("pipe1.gold.gld_despesas_unificadas")
display(df_gold)

In [0]:
# dropar a tabela
spark.sql("DROP TABLE IF EXISTS pipe1.gold.gld_despesas_condominio")